# Callable objects


# Callable


In [4]:
from collections.abc import Awaitable, Callable, Coroutine
from typing import reveal_type


def feeder(
    get_next_item: Callable[[], str],
) -> None:
    ...


def async_query(
    on_success: Callable[[], Awaitable[str]],
) -> None:
    ...


async def on_update() -> str:
    ...


callback1: Callable[[], Coroutine[object, object, str]] = on_update
callback2: Callable[[], Awaitable[str]] = on_update

In [5]:
async_query(on_update)

`Callable` subscription syntax must always be used with exactly two values


In [6]:
x_ellipsis: Callable[..., str]  # Arbitrary argument list

x_without_args: Callable[[], str]  # No arguments

# Two arguments of type `int` and `str`
x_int_str_args: Callable[[int, str], str]

# `Callable` cannot express:

- Functions that take a variadic number of arguments (but not arbitrary)
- Overloaded functions
- Functions that have keyword-only parameters


# `Protocol`


`Protocol` class with a `__call__()` method is able to handle those cases:


In [ ]:
from collections.abc import Iterable
from typing import Protocol


class Combiner(Protocol):
    # Combiner is a callable that takes a variadic number of bytes and an
    # optional `maxlen` keyword-only argument
    def __call__(
        self,
        *vals: bytes,
        maxlen: int | None = None,
    ) -> list[bytes]:
        ...


def combine(
    data: Iterable[bytes],
    combiner: Combiner,
) -> list[bytes]:
    return combiner(*list(data))


def not_combiner_1(
    *vals: bytes,
    maxitems: int | None,  # `maxitems` is not an argument of `Combiner`
) -> list[bytes]:
    ...


combine([], not_combiner_1)  # FAILS


def not_combiner_2(
    *vals: bytes,
    maxlen: int
    | None,  # `maxlen` is an argument of `Combiner`, but it is missing a default
) -> list[bytes]:
    ...


combine([], not_combiner_2)  # FAILS


def not_combiner_3(*vals: bytes) -> list[bytes]:  # `maxlen` is missing
    ...


combine([], not_combiner_3)  # FAILS


def good_combiner(
    *vals: bytes,
    maxlen: int | None = 3,  # `maxlen` is a default argument of `Combiner`
) -> list[bytes]:
    ...


combine([], good_combiner)  # OK

In [ ]:
from collections.abc import Mapping, Sequence


class Employee:
    ...


def notify_by_email(
    employees: Sequence[Employee],
    overrides: Mapping[str, str],
) -> None:
    ...


def first_without_generics(l: Sequence):
    return l[0]


first_element = first_without_generics([1, 2, 3, 4])
# (variable) first_element: Unknown

In [10]:
from typing import TypeVar


T = TypeVar("T")  # Declare type variable "T"


def first(l: Sequence[T]) -> T:
    # Function is generic over the TypeVar "T"
    return l[0]


first_int = first([1, 2, 3, 4])  # (variable) first_int: int

first_str = first(["1", "2", "3", "4"])  # (variable) first_str: str

first_mixed = first([1, "2", 3, "4"])  # (variable) first_mixed: int | str

In [ ]:
x: list[int] = []  # OK

y: list[int] = [1, "1"]  # FAILS

# `list` can accept only one type argument
z: list[int, str] = [1, "1"]  # FAILS

# Mapping can accept two type arguments
m: Mapping[str, str | int] = {}

`tuple` can accept several types in its annotation


In [ ]:
t1: tuple[int, str] = (5, "foo")
t2: tuple[int] = (5,)
t3: tuple[int, str] = (5, "foo")

f1: tuple[int] = (1, 2, 3)  # `c` is a `tuple` of ONE `int`

t4: tuple[int, int, int] = (1, 2, 3)
t5: tuple[int, ...] = (1, 2, 3)
t6: tuple[int, ...] = ()

# If we try to assign a tuple of different types, it fails:
t4 = ("1", "2", "3")

t4 = (4, 5, 6)
t5 = (1, 2, 3)
t5 = ()

t6: tuple[()]
t6 = (2, 3)
t6 = ()

# The type of class objects


In [ ]:
i = 3  # Has type `int`
I = int  # Has type `type[int]`
II = type(I)  # Also has type `type[int]`

`type[C]` is covariant: if `A` is a subtype of `B`, then `type[A]` is a subtype of `type[B]`


In [ ]:
class User:
    ...


class BasicUser(User):
    ...


class ProUser(BasicUser):
    ...


def make_new_user(user_class: type[User]) -> User:
    return user_class()


make_new_user(User)
make_new_user(ProUser)  # `type[ProUser]` is a subtype of `type[User]`
make_new_user(BasicUser)
make_new_user(User())  # expected `type[User]` but got `User`


def return_pro_class_fail(user_class: type[User]) -> type[ProUser]:
    # "type[User]" is incompatible with "type[ProUser]"
    return user_class if type(user_class) is type[ProUser] else ProUser


def return_pro_class_ok(user_class: type[User]) -> type[ProUser]:
    return ProUser

In [ ]:
# Parameter `user_class` accepts subclasses of `type[User]`
a_user_class: type[ProUser] = return_pro_class_ok(BasicUser)
b_user_class: type[ProUser] = return_pro_class_ok(ProUser)


def return_class(user_class: T) -> T:
    return user_class


# Return type can be annotated with a superclass
c_user_class: type[BasicUser] = return_class(BasicUser)
d_user_class: type[BasicUser] = return_class(ProUser)
basic_user: BasicUser = BasicUser()
pro_user: ProUser = ProUser()
# `type[A]` is covariant
pro_user = basic_user
# FAILS: `BasicUser` is not a subtype of `ProUser`, so it cannot be reassigned
pro_user_class = basic_user_class
# FAILS: `type[ProUser]` is not a subtype of `type[BasicUser]`, so it cannot
# be reassigned
basic_user = (
    pro_user
    # OK: `ProUser` is a subtype of `BasicUser`, so reassignment is allowed
)
basic_user_class = pro_user_class


# OK: `type[BasicUser]` is a subtype of `type[ProUser]`, so reassignment is
# allowed
class TeamUser(User):
    ...


def new_non_team_user(user_class: type[BasicUser | ProUser]):
    ...


new_non_team_user(BasicUser)  # OK
new_non_team_user(ProUser)  # OK
new_non_team_user(TeamUser)
# FAILS: Argument of type "type[TeamUser]" cannot be assigned to parameter
# "user_class" of type "type[BasicUser] | type[ProUser]"
new_non_team_user(User)  # Also an error


# `type[Any]` is equivalent to `type`, which is the root of Python's metaclass
# hierarchy.
def is_class(object: type) -> bool:
    return type(object) is type


assert type is type[Any]